In [1]:
from pycaret.classification import *

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

In [3]:
dataset = pd.read_csv('train_data.csv')
dataset

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,a,6,X,3,radiotherapy,Q,F,4.0,86499,23.0,Emergency,Moderate,3,41-50,4144.0,11-20
318434,318435,24,a,1,X,2,anesthesia,Q,E,4.0,325,8.0,Urgent,Moderate,4,81-90,6699.0,31-40
318435,318436,7,a,4,X,3,gynecology,R,F,4.0,125235,10.0,Emergency,Minor,3,71-80,4235.0,11-20
318436,318437,11,b,2,Y,3,anesthesia,Q,D,3.0,91081,8.0,Trauma,Minor,5,11-20,3761.0,11-20


In [4]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))
data_unseen.to_csv('new_data_healthcare.csv')

Data for Modeling: (302516, 18)
Unseen Data For Predictions: (15922, 18)


In [5]:
data

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,148063,1,d,10,Y,2,gynecology,S,B,3.0,69152,8.0,Emergency,Extreme,3,61-70,5659.0,71-80
1,83846,23,a,6,X,3,TB & Chest disease,R,F,2.0,112794,2.0,Trauma,Extreme,4,41-50,5190.0,71-80
2,95516,21,c,3,Z,2,gynecology,S,A,2.0,129679,5.0,Emergency,Minor,4,71-80,4828.0,51-60
3,155942,26,b,2,Y,2,anesthesia,Q,D,2.0,15422,2.0,Trauma,Extreme,2,61-70,8892.0,21-30
4,15727,12,a,9,Y,2,gynecology,R,B,3.0,31653,8.0,Trauma,Moderate,3,31-40,5104.0,21-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302511,170575,23,a,6,X,3,radiotherapy,Q,F,3.0,16484,6.0,Urgent,Minor,4,61-70,5046.0,31-40
302512,204846,23,a,6,X,6,gynecology,Q,F,4.0,8362,2.0,Trauma,Moderate,4,51-60,4448.0,21-30
302513,282092,29,a,4,X,3,gynecology,R,F,2.0,109367,8.0,Urgent,Moderate,2,21-30,4838.0,21-30
302514,259884,14,a,1,X,4,gynecology,S,E,2.0,32105,NaN,Emergency,Minor,4,61-70,4376.0,21-30


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302516 entries, 0 to 302515
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            302516 non-null  int64  
 1   Hospital_code                      302516 non-null  int64  
 2   Hospital_type_code                 302516 non-null  object 
 3   City_Code_Hospital                 302516 non-null  int64  
 4   Hospital_region_code               302516 non-null  object 
 5   Available Extra Rooms in Hospital  302516 non-null  int64  
 6   Department                         302516 non-null  object 
 7   Ward_Type                          302516 non-null  object 
 8   Ward_Facility_Code                 302516 non-null  object 
 9   Bed Grade                          302407 non-null  float64
 10  patientid                          302516 non-null  int64  
 11  City_Code_Patient                  2982

In [7]:
exp_clf101 =setup(data, target = 'Stay',
            ignore_features = ['case_id','patientid'],
 #           silent=True,
            categorical_features=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age'],
            numeric_features=['Hospital_code', 'City_Code_Hospital','Available Extra Rooms in Hospital','Bed Grade', 'City_Code_Patient', 'Visitors with Patient', 'Admission_Deposit'],
            session_id=543)

,Description,Value
0,Session id,543
1,Target,Stay
2,Target type,Multiclass
3,Target mapping,"0-10: 0, 11-20: 1, 21-30: 2, 31-40: 3, 41-50: 4, 51-60: 5, 61-70: 6, 71-80: 7, 81-90: 8, 91-100: 9, More than 100 Days: 10"
4,Original data shape,"(302516, 18)"
5,Transformed data shape,"(302516, 51)"
6,Transformed train set shape,"(211761, 51)"
7,Transformed test set shape,"(90755, 51)"
8,Ignore features,2
9,Numeric features,7


In [8]:
# compare models
top5 = compare_models(n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.4231,0.7602,0.4231,0.4063,0.3886,0.2595,0.2664,36.4620
gbc,Gradient Boosting Classifier,0.4137,0.7493,0.4137,0.3948,0.3708,0.2470,0.2551,1.8620
lda,Linear Discriminant Analysis,0.3789,0.7151,0.3789,0.3471,0.3238,0.1952,0.2037,0.6320
ada,Ada Boost Classifier,0.3745,0.6687,0.3745,0.3286,0.3382,0.1970,0.2016,0.9780
ridge,Ridge Classifier,0.3545,0.0000,0.3545,0.2809,0.2808,0.1491,0.1611,0.5740
lr,Logistic Regression,0.3498,0.6772,0.3498,0.2905,0.3044,0.1533,0.1579,4.9940
dt,Decision Tree Classifier,0.2963,0.5682,0.2963,0.2978,0.2970,0.1365,0.1365,1.1220
nb,Naive Bayes,0.2934,0.6546,0.2934,0.3021,0.2816,0.1343,0.1362,0.6870
dummy,Dummy Classifier,0.2744,0.5000,0.2744,0.0753,0.1181,0.0000,0.0000,0.8580
knn,K Neighbors Classifier,0.2666,0.5666,0.2666,0.2435,0.2437,0.0622,0.0633,31.9550


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
# tune models
tuned_top5 = [tune_model(i) for i in top5]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:46:29
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.691122 seconds.
You can set `force_row_wise=true` to remove the over

In [ ]:
# ensemble models
bagged_top5 = [ensemble_model(i) for i in tuned_top5]

In [ ]:
# blend models
blender = blend_models(estimator_list = top5)

In [ ]:
# stack models
stacker = stack_models(estimator_list = top5)

In [ ]:
# automl
best = automl(optimize = 'F1')
print(best)